In [1]:
# import neccessary packages
import random
import math
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict
import time
from scipy.spatial.distance import cdist
import pygad
from tqdm.notebook import tqdm_notebook

In [2]:
tilbury_df = pd.read_excel('C:/Users/visha/Downloads/aco/tilbury_data_new.xlsx', sheet_name='final')
tilbury_df['n1'] = tilbury_df['n1'].astype(str)
tilbury_df['n2'] = tilbury_df['n2'].astype(str)
tilbury_df['edge_key'] = tilbury_df['n1']+"-"+tilbury_df['n2']
tilbury_df['weight'] = tilbury_df['weight'].astype(float)
tilbury_df['edge_stat'] = tilbury_df['edge_stat'].astype(str)

tt = pd.DataFrame(tilbury_df[['n1','n2','edge_stat','edge_key','weight']])

target_segments = tt[tt["edge_stat"] == "target"]["edge_key"].tolist()
normal_segments = tt[tt["edge_stat"] == "normal"]["edge_key"].tolist()

modified = []
for i in target_segments:
    for j in i.split("-"):
        modified.append(j)

modified = set(modified.copy())
modified = list(modified)

cols = ['n1', 'n2', 'weight']
df_graph = tt[cols]
g = nx.DiGraph()
g.add_weighted_edges_from(df_graph.values)

print("Number of Nodes:", g.number_of_nodes())
print("Number of Edges:", g.number_of_edges())

Number of Nodes: 1199
Number of Edges: 2663


In [3]:
def find_nodes_to_remove(graph, target_nodes = None):
    to_remove = []
    for node in graph.nodes:
        in_nodes = set(graph.predecessors(node))
        out_nodes = set(graph.successors(node))
        # Condition A: out_nodes == in_nodes
        # Condition B: out_nodes is empty
        if target_nodes:
            if ((len(out_nodes)==1 and out_nodes == in_nodes) or len(out_nodes) == 0) and (node not in target_nodes) :
                to_remove.append(node)
        else:
            if (len(out_nodes)==1 and out_nodes == in_nodes) or len(out_nodes) == 0:
                to_remove.append(node)
    return to_remove

# --- 3) REMOVE NODES ITERATIVELY UNTIL NO MORE MATCH ---
while True:
    removable = find_nodes_to_remove(g, target_nodes = modified)
    if not removable:
        break
    g.remove_nodes_from(removable)
    print("Removed:", removable)

print("Remaining Nodes:", len(list(g.nodes)))
print("Remaining Edges:", len(list(g.edges)))

Removed: ['828978312371498808', '7526152829053408261', '16816866342600266120', '4856429617603906865', '15436324000118763095', '10421480741551812636', '11888290701321009334', '12126645967899173579', '7670611776292009507', '9315010370675346771', '5000088206271940899', '15055828116576676654', '2668463261307506215', '14469086530543940450', '1434189148877398403', '17263110374276783671', '692390352451657769', '3997259477668285087', '5910713753442694368', '7154067901523233197', '1478327180679980114', '14862439878145419229', '384933875519181471', '8204511724585490491', '12959694865737371104', '7880385517297410923', '4035250460769237030', '11572648929703594496', '12274079678168499723', '11030890099212226135', '7513174043165738024', '10070294354038688599', '16700777385767785721', '6238885609185014847', '5050172923295377706', '13324441122801468508', '42223450892923563', '224096620785378170', '17357357445316339818', '4508094162487989285', '17802594430593197102', '7871156652650675225', '33303077365

In [4]:
g['7956601085024358655']

AtlasView({'8914508907649566259': {'weight': 98.20494509}})

In [58]:
g['8914508907649566259']

AtlasView({'748558481888905982': {'weight': 35.11865413}, '7956601085024358655': {'weight': 98.20494509}, '6571401944664505419': {'weight': 92.68272189}})

In [65]:
modified == '7956601085024358655'
for x in modified:
    if x == '7956601085024358655':
        print(x)

7956601085024358655


In [5]:
# Creating distance matrix from target to target nodes

# Initialize data structures for results
dynamic_targets = modified.copy()
distances_df = pd.DataFrame(index=dynamic_targets, columns=dynamic_targets, dtype=float)
path_details = []  # To store from_node, to_node, distance, path


def calculate_all_paths():
    total_pairs = len(dynamic_targets) * (len(dynamic_targets) - 1)  # Total pairs excluding self-loops
    progress_bar = tqdm_notebook(total=total_pairs, desc="Calculating paths", unit="pair")

    for s in dynamic_targets:
        for t in dynamic_targets:
            if s != t:  # No need to compute paths to itself
                try:
                    # Compute shortest path
                    path = nx.shortest_path(g, source=s, target=t, weight="weight", method="dijkstra")
                    
                    # Calculate path length (sum of weights along the path)
                    path_length = sum(
                        g[u][v]["weight"] for u, v in zip(path[:-1], path[1:])
                    )

                    # Update distances_df
                    distances_df.loc[s, t] = path_length

                    # Append details for path_details_df
                    path_details.append({
                        "from_node": s,
                        "to_node": t,
                        "distance": path_length,
                        "path": path
                    })
                except nx.NetworkXNoPath:
                    # If no path, leave distance_df as NaN (default)
                    continue
                finally:
                    # Update the progress bar
                    progress_bar.update(1)

    progress_bar.close()


def save_results():
    # Convert path details into a DataFrame
    path_details_df = pd.DataFrame(path_details)

    print("Distances DataFrame:")
    print(distances_df)
    print("\nPath Details DataFrame:")
    print(path_details_df)

    return distances_df, path_details_df


# Example usage:
calculate_all_paths()
distances_df, path_details_df = save_results()


Calculating paths:   0%|          | 0/511940 [00:00<?, ?pair/s]

Distances DataFrame:
                      5669140911938668079  522192584878887089  \
5669140911938668079                   NaN         4395.654996   
522192584878887089            3797.156071                 NaN   
17244616307975579376          3490.266836         4209.142508   
3753353808586198144           1496.265786         3387.543993   
5892184120801996297           1726.698419         3349.315931   
...                                   ...                 ...   
13379508616222364625          2524.851237         3911.163371   
10422649938504079974          1917.731552         3495.860278   
7006124127845995444           1692.696929         3789.970837   
8151528648277500274           1581.125706         3604.614426   
9893121337639984162           2477.565976         3196.441648   

                      17244616307975579376  3753353808586198144  \
5669140911938668079            3206.495322          1008.111003   
522192584878887089             3412.170038          3731.971614 

In [6]:
print(path_details_df.head())
print((path_details_df.shape))

print(distances_df.head())
print((distances_df.shape))


             from_node               to_node     distance  \
0  5669140911938668079    522192584878887089  4395.654996   
1  5669140911938668079  17244616307975579376  3206.495322   
2  5669140911938668079   3753353808586198144  1008.111003   
3  5669140911938668079   5892184120801996297  1505.180893   
4  5669140911938668079   5632726525166839536  2310.830668   

                                                path  
0  [5669140911938668079, 2375399834296968840, 696...  
1  [5669140911938668079, 2375399834296968840, 696...  
2  [5669140911938668079, 2375399834296968840, 696...  
3  [5669140911938668079, 2375399834296968840, 696...  
4  [5669140911938668079, 2375399834296968840, 696...  
(502019, 4)
                      5669140911938668079  522192584878887089  \
5669140911938668079                   NaN         4395.654996   
522192584878887089            3797.156071                 NaN   
17244616307975579376          3490.266836         4209.142508   
3753353808586198144           1

In [10]:
# target_distances = distances_df.set_index(list(distances_df)[0])
# print(target_distances.head())
target_distances = distances_df.copy()
# Convert row labels (index) to strings
target_distances.index = target_distances.index.astype(str)

print("Unique Row Labels:", target_distances.index.nunique())
print("Unique Column Labels:", target_distances.columns.nunique())

# Check for mismatches
assert set(target_distances.index) == set(target_distances.columns), "Row and column labels do not match!"

print("Row Labels Type:", type(target_distances.index[0]))
print("Column Labels Type:", type(target_distances.columns[0]))

Unique Row Labels: 716
Unique Column Labels: 716
Row Labels Type: <class 'str'>
Column Labels Type: <class 'str'>


In [11]:
G = nx.DiGraph()

# Add edges with conditions
for source in target_distances.index:
    for target in target_distances.columns:
        distance = target_distances.loc[source, target]
        if pd.notna(distance) and distance != 0 and np.isfinite(distance):
            G.add_edge(source, target, weight=distance)

# Print counts
print("Number of Nodes:", G.number_of_nodes())
print("Number of Edges:", G.number_of_edges())

Number of Nodes: 716
Number of Edges: 502019


In [18]:
def initialize_pheromone(graph, init_pheromone, node_mapping, target_df, extra_pheromone):
    """
    Initialize pheromone level for every edge in the graph.
    Optimized using NumPy arrays.
    
    Parameters:
    - graph: The graph structure (not used directly in this function but kept for context).
    - init_pheromone: The initial pheromone level for all edges.
    - node_mapping: A dictionary mapping node identifiers to indices in the pheromone matrix.
    - target_df: A DataFrame containing columns 'n1' and 'n2' representing target edges.
    - extra_pheromone: The additional pheromone to assign to target edges.
    
    Returns:
    - pheromone: A 2D NumPy array representing the pheromone levels for each edge.
    """
    num_nodes = len(node_mapping)
    pheromone = np.full((num_nodes, num_nodes), init_pheromone, dtype=np.float32)
    
    # Add extra pheromone for edges in target_df
    for _, row in target_df.iterrows():
        n1 = row['n1']
        n2 = row['n2']
        if n1 in node_mapping and n2 in node_mapping:
            i, j = node_mapping[n1], node_mapping[n2]
            pheromone[i, j] += extra_pheromone
    
    return pheromone

def create_ants(n_ants, graph, min_visit_percentage=0.6, node_mapping=None):
    """
    Create ants with tracking of minimum required visits.
    Optimized with NumPy-friendly structures.
    """
    nodes_list = np.array(list(node_mapping.keys()))  # Use original nodes
    num_nodes = len(nodes_list)
    min_required_visits = int(num_nodes * min_visit_percentage)

    ants = {
        'path': [[] for _ in range(n_ants)],
        'visited_nodes': [set() for _ in range(n_ants)],
        'path_edges': [[] for _ in range(n_ants)],
        'stack': [[] for _ in range(n_ants)],
        'visit_count': np.zeros((n_ants, num_nodes), dtype=int),
        'min_required_visits': min_required_visits
    }

    for i in range(n_ants):
        start_node = random.choice(nodes_list)
        ants['path'][i].append(start_node)
        ants['visited_nodes'][i].add(start_node)
        ants['stack'][i].append(start_node)
        ants['visit_count'][i, node_mapping[start_node]] = 1  # Use mapped index

    return ants

def select_next_node(ant_idx, ants, graph, pheromone, heuristic, alpha, unvisited_only=True, max_visits=10, node_mapping=None):
    """
    Select the next node using cached heuristic values and ACO probability rules.
    """
    current_node = ants['path'][ant_idx][-1]
    current_node_idx = node_mapping[current_node]

    neighbors = list(graph.successors(current_node))
    if unvisited_only:
        neighbors = [n for n in neighbors if n not in ants['visited_nodes'][ant_idx]]
    else:
        neighbors = [n for n in neighbors if ants['visit_count'][ant_idx, node_mapping[n]] < max_visits]

    if not neighbors:
        return None

    neighbor_indices = [node_mapping[n] for n in neighbors]

    # Use cached heuristic values
    tau = pheromone[current_node_idx, neighbor_indices] ** alpha
    eta = heuristic[current_node_idx, neighbor_indices]
    weights = tau * eta

    total = weights.sum()
    if total <= 1e-12:
        return None

    # Roulette wheel selection
    probabilities = weights / total
    return np.random.choice(neighbors, p=probabilities)

def precompute_heuristic(graph, node_mapping, beta=2.0):
    """
    Precompute heuristic values (1 / weight ** beta) for all edges in the graph.
    Returns a 2D NumPy array indexed by node indices.
    """
    num_nodes = len(node_mapping)
    heuristic = np.zeros((num_nodes, num_nodes), dtype=np.float32)

    for u, v, data in graph.edges(data=True):
        u_idx = node_mapping[u]
        v_idx = node_mapping[v]
        weight = data.get('weight', 1.0)
        if weight > 0:
            heuristic[u_idx, v_idx] = (5.0 / weight) ** beta #50/weight
        else:
            heuristic[u_idx, v_idx] = 0.0  # Handle zero or invalid weights safely

    return heuristic


def move_ant(ant_idx, ants, next_node, node_mapping):
    """
    Optimized move operation for ants using node_mapping.
    """
    current_node = ants['path'][ant_idx][-1]
    ants['path'][ant_idx].append(next_node)
    ants['visited_nodes'][ant_idx].add(next_node)
    ants['path_edges'][ant_idx].append((current_node, next_node))
    ants['stack'][ant_idx].append(next_node)

    # Update visit count using mapped indices
    next_node_idx = node_mapping[next_node]
    ants['visit_count'][ant_idx, next_node_idx] += 1

def evaporate_pheromone(pheromone, evaporation_rate):
    """
    Optimized pheromone evaporation using NumPy.
    """
    pheromone *= (1.0 - evaporation_rate)

def compute_path_cost(path_edges, graph, node_mapping):
    """
    Compute the total cost of a path using the graph weights.
    Uses node_mapping to translate nodes to their indices.
    """
    total_cost = 0.0
    for u, v in path_edges:
        total_cost += graph[u][v].get('weight', 1.0)
    return total_cost


In [23]:
# def process_top_ants(
#     ants, graph, pheromone, node_mapping, compute_path_cost,
#     target_df, top_percent=0.3, initial_filter_percent=0.5, deposit_factor=1.0, virtual_pheromone_weight=0.1
# ):
#     """
#     Process top-performing ants and return a sorted DataFrame with additional metrics.
#     """
#     target_edges = set(zip(target_df['n1'], target_df['n2']))
#     target_cost_map = dict(zip(target_edges, target_df['weight']))
    
#     rows = []
#     for i in range(len(ants['path'])):
#         path_edges = ants['path_edges'][i]
#         distinct_target_edges = set(path_edges) & target_edges
#         distinct_target_count = len(distinct_target_edges)
#         distinct_target_cost_sum = sum(target_cost_map[edge] for edge in distinct_target_edges)
#         total_cost = compute_path_cost(path_edges, graph, node_mapping)
#         target_cost_ratio = distinct_target_cost_sum / total_cost if total_cost > 0 else 0
        
#         rows.append({
#             "index": i,
#             "path": ants['path'][i],
#             "path_edges": path_edges,
#             "visited_nodes": ants['visited_nodes'][i],
#             "distinct_target_count": distinct_target_count,
#             "distinct_target_cost": distinct_target_cost_sum,
#             "target_cost_ratio": target_cost_ratio,
#             "total_cost": total_cost,
#         })

#     # Create a DataFrame
#     df = pd.DataFrame(rows)

#     # First filter: Select top 50% ants based on distinct_target_count
#     n_initial_filter = max(1, int(len(df) * initial_filter_percent))
#     df = df.sort_values(by=["distinct_target_count"], ascending=False).head(n_initial_filter)

#     # Second filter: Select top top_percent ants based on target_cost_ratio
#     n_top = max(1, int(len(df) * top_percent))
#     top_df = df.sort_values(by=["target_cost_ratio"], ascending=False).head(n_top)

#     # Deposit pheromone for top ants
#     visited_by_top_ants = set()
#     for _, row in top_df.iterrows():
#         path_edges = row['path_edges']
#         path_len = len(row['path']) - 1
#         if path_len < 1:
#             continue
#         contribution = deposit_factor / path_len
#         for u, v in path_edges:
#             u_idx = node_mapping[u]
#             v_idx = node_mapping[v]
#             pheromone[u_idx, v_idx] += contribution
#         visited_by_top_ants.update(row['visited_nodes'])

#     # Reweight pheromones for unvisited nodes
#     all_nodes = set(graph.nodes)
#     unvisited_nodes = all_nodes - visited_by_top_ants
#     for node in unvisited_nodes:
#         idx = node_mapping[node]
#         pheromone[idx, :] += virtual_pheromone_weight
#         pheromone[:, idx] += virtual_pheromone_weight

#     return top_df


In [24]:
def process_top_ants(
    ants, graph, pheromone, node_mapping, compute_path_cost,
    target_df, top_percent=0.3, deposit_factor=1.0, virtual_pheromone_weight=0.1
):
    """
    Process top-performing ants and return a sorted DataFrame with additional metrics.
    """
    target_edges = set(zip(target_df['n1'], target_df['n2']))
    target_cost_map = dict(zip(target_edges, target_df['weight']))
    
    rows = []
    for i in range(len(ants['path'])):
        path_edges = ants['path_edges'][i]
        distinct_target_edges = set(path_edges) & target_edges
        distinct_target_count = len(distinct_target_edges)
        distinct_target_cost_sum = sum(target_cost_map[edge] for edge in distinct_target_edges)
        total_cost = compute_path_cost(path_edges, graph, node_mapping)
        target_cost_ratio = distinct_target_cost_sum / total_cost if total_cost > 0 else 0
        
        combined_metric = distinct_target_count * target_cost_ratio
        
        rows.append({
            "index": i,
            "path": ants['path'][i],
            "path_edges": path_edges,
            "visited_nodes": ants['visited_nodes'][i],
            "distinct_target_count": distinct_target_count,
            "distinct_target_cost": distinct_target_cost_sum,
            "target_cost_ratio": target_cost_ratio,
            "total_cost": total_cost,
            "combined_metric": combined_metric
        })

    # Create a DataFrame
    df = pd.DataFrame(rows)

    # Sort by combined_metric and select top top_percent ants
    n_top = max(1, int(len(df) * top_percent))
    top_df = df.sort_values(by=["combined_metric"], ascending=False).head(n_top)

    # Deposit pheromone for top ants
    visited_by_top_ants = set()
    for _, row in top_df.iterrows():
        path_edges = row['path_edges']
        path_len = len(row['path']) - 1
        if path_len < 1:
            continue
        contribution = deposit_factor / path_len
        for u, v in path_edges:
            u_idx = node_mapping[u]
            v_idx = node_mapping[v]
            pheromone[u_idx, v_idx] += contribution
        visited_by_top_ants.update(row['visited_nodes'])

    # Reweight pheromones for unvisited nodes
    all_nodes = set(graph.nodes)
    unvisited_nodes = all_nodes - visited_by_top_ants
    for node in unvisited_nodes:
        idx = node_mapping[node]
        pheromone[idx, :] += virtual_pheromone_weight
        pheromone[:, idx] += virtual_pheromone_weight

    return top_df


In [14]:
# def run_aco(
#     graph,
#     target_df,
#     extra_pheromone = 0.01,
#     n_ants=10,
#     alpha=1.0,
#     beta=2.0,
#     evaporation_rate=0.5,
#     init_pheromone=0.01,
#     iterations=10,
#     unvisited_only=True,
#     max_visits=10,
#     min_visit_percentage=0.6,
#     top_percent=0.3,
#     virtual_pheromone_weight=0.1
# ):
#     """
#     Main ACO loop with heuristic caching for improved performance.
#     """
#     node_mapping = {node: idx for idx, node in enumerate(graph.nodes)}
#     reverse_mapping = {idx: node for node, idx in node_mapping.items()}
#     pheromone = initialize_pheromone(graph, init_pheromone, node_mapping,target_df, extra_pheromone)

#     # Precompute heuristic values
#     heuristic = precompute_heuristic(graph, node_mapping, beta)

#     global_best_path = None
#     global_best_cost = float('inf')
#     iteration_best_paths = []
#     iteration_best_costs = []
#     top_ants_data = []
#     top_df_list = []

#     total_start_time = time.time()

#     for it in range(iterations):
#         ants = create_ants(n_ants, graph, min_visit_percentage=min_visit_percentage, node_mapping=node_mapping)
#         unvisited_only = True  # it < (0.75 * iterations)
#         for ant_idx in range(n_ants):
#             while True:  # len(ants['visited_nodes'][ant_idx]) < ants['min_required_visits']:
#                 next_node = select_next_node(
#                     ant_idx, ants, graph, pheromone, heuristic, alpha,
#                     unvisited_only=unvisited_only, max_visits=max_visits, node_mapping=node_mapping
#                 )
#                 if next_node is not None:
#                     move_ant(ant_idx, ants, next_node, node_mapping)  # Pass node_mapping here
#                 else:
#                     break

#         # Get sorted DataFrame of top ants
#         top_df = process_top_ants(
#             ants, graph, pheromone, node_mapping, compute_path_cost,
#             target_df, top_percent=top_percent, deposit_factor=1.0, virtual_pheromone_weight=virtual_pheromone_weight
#         )

#         top_df["iteration"] = it + 1
#         top_df_list.append(top_df)

#         # Extract top paths and costs
#         top_paths = top_df["path"].tolist()
#         top_costs = top_df["total_cost"].tolist()

#         evaporate_pheromone(pheromone, evaporation_rate)
#         pheromone_list = []
#         pheromone_list.append(pheromone)

#         # Identify the iteration best
#         iteration_best_row = top_df.iloc[0]
#         iteration_best_path = iteration_best_row["path"]
#         iteration_best_cost = iteration_best_row["total_cost"]
#         iteration_best_distinct_target_count = iteration_best_row["distinct_target_count"]
#         iteration_best_distinct_target_cost = iteration_best_row["distinct_target_cost"]

#         # Translate nodes in iteration_best_path to original node labels using node_mapping and reverse_mapping
#         iteration_best_paths.append([reverse_mapping[node_mapping[node]] for node in iteration_best_path])
#         iteration_best_costs.append(iteration_best_cost)

#         # Update global best if the iteration best is better
#         distinct_nodes_iteration = len(set(iteration_best_path))
#         distinct_nodes_global = len(set(global_best_path or []))

#         if distinct_nodes_iteration > distinct_nodes_global:
#             global_best_path = iteration_best_path
#             global_best_cost = iteration_best_cost
#         elif distinct_nodes_iteration == distinct_nodes_global:
#             if iteration_best_cost < global_best_cost:
#                 global_best_path = iteration_best_path
#                 global_best_cost = iteration_best_cost

#         # Store top ants' data for this iteration
#         top_ants_data.append({
#             "iteration": it + 1,
#             "top_n_paths": [[reverse_mapping[node_mapping[node]] for node in path] for path in top_paths],
#             "top_n_cost": top_costs
#         })

#         print(f"Iteration {it+1}/{iterations} - Iteration Best Path: {len(set(iteration_best_path))} nodes, "
#               f"Cost: {iteration_best_cost:.4f}, Distinct Target Edges: {iteration_best_distinct_target_count}, "
#               f"Distinct Target Cost: {iteration_best_distinct_target_cost:.4f}, "
#               f"Global Best Cost: {global_best_cost:.4f}")

#     total_time = time.time() - total_start_time

#     print(f"Total time: {total_time:.4f}s")

#     # Create the DataFrame from top_ants_data
#     top_ants_df = pd.DataFrame(top_ants_data)
#     full_top_df = pd.concat(top_df_list, ignore_index=True)

#     return {
#         "global_best_path": [reverse_mapping[node_mapping[node]] for node in global_best_path],
#         "global_best_cost": global_best_cost,
#         "iteration_best_paths": iteration_best_paths,
#         "iteration_best_costs": iteration_best_costs,
#         "full_top_df": full_top_df,
#         "pheromone_matrix": pheromone_list
#     }



In [27]:
def run_aco(
    graph,
    target_df,
    extra_pheromone=0.01,
    n_ants=10,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.5,
    init_pheromone=0.01,
    iterations=10,
    unvisited_only=True,
    max_visits=10,
    min_visit_percentage=0.6,
    top_percent=0.3,
    virtual_pheromone_weight=0.1
):
    """
    Main ACO loop with heuristic caching for improved performance.
    """
    node_mapping = {node: idx for idx, node in enumerate(graph.nodes)}
    reverse_mapping = {idx: node for node, idx in node_mapping.items()}
    pheromone = initialize_pheromone(graph, init_pheromone, node_mapping, target_df, extra_pheromone)

    # Precompute heuristic values
    heuristic = precompute_heuristic(graph, node_mapping, beta)

    global_best_path = None
    global_best_cost = float('inf')
    iteration_best_paths = []
    iteration_best_costs = []
    top_ants_data = []
    top_df_list = []

    total_start_time = time.time()

    for it in range(iterations):
        ants = create_ants(n_ants, graph, min_visit_percentage=min_visit_percentage, node_mapping=node_mapping)
        unvisited_only = True  # it < (0.75 * iterations)
        for ant_idx in range(n_ants):
            while True:
                next_node = select_next_node(
                    ant_idx, ants, graph, pheromone, heuristic, alpha,
                    unvisited_only=unvisited_only, max_visits=max_visits, node_mapping=node_mapping
                )
                if next_node is not None:
                    move_ant(ant_idx, ants, next_node, node_mapping)
                else:
                    break

        # Get sorted DataFrame of top ants
        top_df = process_top_ants(
            ants, graph, pheromone, node_mapping, compute_path_cost,
            target_df, top_percent=top_percent, deposit_factor=1.0, virtual_pheromone_weight=virtual_pheromone_weight
        )

        top_df["iteration"] = it + 1
        top_df_list.append(top_df)

        # Select the top ant based on the number of target edges
        top_ant_row = top_df.sort_values(by=["distinct_target_count"], ascending=False).iloc[0]

        # Extract metrics for the top ant
        top_ant_metrics = {
            "total_cost": top_ant_row["total_cost"],
            "distinct_target_count": top_ant_row["distinct_target_count"],
            "target_cost_ratio": top_ant_row["target_cost_ratio"],
            "combined_metric": top_ant_row["distinct_target_count"] * top_ant_row["target_cost_ratio"]
        }

        print(f"Iteration {it+1}/{iterations} - Top Ant Metrics:  Total Cost: {top_ant_metrics['total_cost']:.4f}  Distinct Target Edges: {top_ant_metrics['distinct_target_count']} Target Cost Ratio: {top_ant_metrics['target_cost_ratio']:.4f} Combined Metric: {top_ant_metrics['combined_metric']:.4f}")

        # Update global best if needed
        if top_ant_row["distinct_target_count"] > (len(set(global_best_path or [])) if global_best_path else 0):
            global_best_path = top_ant_row["path"]
            global_best_cost = top_ant_row["total_cost"]

        evaporate_pheromone(pheromone, evaporation_rate)

    total_time = time.time() - total_start_time
    print(f"Total time: {total_time:.4f}s")

    # Create the DataFrame from top_ants_data
    full_top_df = pd.concat(top_df_list, ignore_index=True)

    return {
        "global_best_path": [reverse_mapping[node_mapping[node]] for node in global_best_path],
        "global_best_cost": global_best_cost,
        "full_top_df": full_top_df,
        "pheromone_matrix": pheromone
    }


In [28]:
# Main ACO iteration path - lomgest possible path

result = run_aco(
    graph=G,
    target_df= tt[tt["edge_stat"] == "target"],
    extra_pheromone= 0.5,
    n_ants=200,
    alpha=2.0,
    beta=6.0,
    evaporation_rate=0.2,
    init_pheromone=0.5,
    iterations=10,
    unvisited_only=True,
    max_visits=15,
    min_visit_percentage =1.5,
    top_percent=0.25,
    virtual_pheromone_weight=0.3
)


# print("Global Best Path:", result["global_best_path"])
print("Global Best Cost:", result["global_best_cost"])
# print("Iteration Best Paths:", result["iteration_best_paths"])
# print("Iteration Best Costs:", result["iteration_best_costs"])


Iteration 1/10 - Top Ant Metrics:  Total Cost: 38578.7895  Distinct Target Edges: 179 Target Cost Ratio: 0.3134 Combined Metric: 56.1070
Iteration 2/10 - Top Ant Metrics:  Total Cost: 28295.6025  Distinct Target Edges: 166 Target Cost Ratio: 0.3762 Combined Metric: 62.4527
Iteration 3/10 - Top Ant Metrics:  Total Cost: 22201.0953  Distinct Target Edges: 143 Target Cost Ratio: 0.4107 Combined Metric: 58.7279
Iteration 4/10 - Top Ant Metrics:  Total Cost: 24194.3116  Distinct Target Edges: 151 Target Cost Ratio: 0.3815 Combined Metric: 57.6041
Iteration 5/10 - Top Ant Metrics:  Total Cost: 25550.1285  Distinct Target Edges: 148 Target Cost Ratio: 0.3451 Combined Metric: 51.0815
Iteration 6/10 - Top Ant Metrics:  Total Cost: 22698.3190  Distinct Target Edges: 141 Target Cost Ratio: 0.3938 Combined Metric: 55.5308
Iteration 7/10 - Top Ant Metrics:  Total Cost: 20713.3644  Distinct Target Edges: 136 Target Cost Ratio: 0.4338 Combined Metric: 58.9994
Iteration 8/10 - Top Ant Metrics:  Total 

In [34]:
full_top_df = result["full_top_df"]
# best_df_1 = full_top_df[full_top_df["distinct_target_count"] == full_top_df["distinct_target_count"].max()]

mini_df = full_top_df[full_top_df["combined_metric"] > 55]
mini_df_final = mini_df[mini_df['distinct_target_count'] == mini_df['distinct_target_count'].max()]
print(mini_df_final)
bp_1 = mini_df_final["path"].to_list()[0]

   index                                               path  \
2     86  [7612750293809454279, 1277434071787471965, 182...   

                                          path_edges  \
2  [(7612750293809454279, 1277434071787471965), (...   

                                       visited_nodes  distinct_target_count  \
2  {17244616307975579376, 5892184120801996297, 43...                    179   

   distinct_target_cost  target_cost_ratio    total_cost  combined_metric  \
2          12092.402716           0.313447  38578.789543        56.106998   

   iteration  
2          1  


In [35]:
def construct_full_path(final_path, path_details_df):
    """
    Constructs the full path list based on the final path list and path details DataFrame.

    Parameters:
    - final_path (list): The list of nodes representing the main path sequence (e.g., ["A", "D", "F"]).
    - path_details_df (pd.DataFrame): DataFrame containing `from_node`, `to_node`, and `path`.
      Each row represents a subpath with nodes connecting `from_node` to `to_node`.

    Returns:
    - list: Full path list combining all subpaths from path_details_df.

    Raises:
    - ValueError: If a subpath for a pair of nodes is not found in path_details_df.
    """
    full_path = []

    # Loop through consecutive nodes in the final_path
    for i in range(len(final_path) - 1):
        current_node = final_path[i]
        next_node = final_path[i + 1]
        
        # Find the subpath in the DataFrame
        subpath_row = path_details_df[
            (path_details_df["from_node"] == current_node) &
            (path_details_df["to_node"] == next_node)
        ]
        
        if not subpath_row.empty:
            subpath = subpath_row.iloc[0]["path"]
            # Add the subpath to the full path, avoiding duplicate nodes
            if full_path:
                full_path.extend(subpath[1:])  # Skip the first node of the subpath
            else:
                full_path.extend(subpath)  # Add the entire first subpath
        else:
            raise ValueError(f"No subpath found for {current_node} -> {next_node}")

    return full_path

full_path_1 = construct_full_path(final_path=result['global_best_path'], path_details_df=path_details_df)

def create_ordered_dataframe(full_path, original_df):
    """
    Creates a new DataFrame with rows from `original_df` ordered according to the sequence in `full_path`.

    Parameters:
    - full_path (list): List of nodes defining the order (e.g., ["A", "D", "F"]).
    - original_df (pd.DataFrame): Original DataFrame with columns `from_node`, `to_node`, `weight`, and `source`.

    Returns:
    - pd.DataFrame: New DataFrame with rows ordered according to `full_path`.
    """
    # Create an empty list to store matching rows
    ordered_rows = []

    # Iterate through consecutive pairs in full_path
    for i in range(len(full_path) - 1):
        current_node = full_path[i]
        next_node = full_path[i + 1]
        
        # Find the corresponding row in the original DataFrame
        row = original_df[
            (original_df["n1"] == current_node) &
            (original_df["n2"] == next_node)
        ]
        
        if not row.empty:
            ordered_rows.append(row.iloc[0])  # Append the first matching row
        else:
            raise ValueError(f"No row found for {current_node} -> {next_node}")

    # Create a new DataFrame from the ordered rows
    new_df = pd.DataFrame(ordered_rows)
    return new_df

full_path_df = create_ordered_dataframe(full_path=full_path_1, original_df=tilbury_df)

In [20]:
# def remove_repeated_loops_keep_one(path):
#     """
#     Removes immediately repeated sub-sequences in the path,
#     keeping only one copy of that sub-sequence.
#     """
#     if not path:
#         return path
    
#     # We'll do a simple scan, looking for repeats of length 2 or more
#     # This method will handle the typical patterns like X,Y repeated multiple times (X,Y,X,Y,...).
    
#     result = []
#     i = 0
#     while i < len(path):
#         # For each i, search forward for the next repeated pattern
#         # We'll do a naive approach: detect the smallest repeating sub-sequence around i
#         # and skip the duplicates
        
#         # Step 1: Attempt to find any repeated sub-sequence starting at i
#         # For instance, if path[i : i+2] == path[i+2 : i+4], then we have a sub-sequence repeat
#         repeat_len = 1  # length of repeating pattern
#         found_repeat = False
        
#         # Try all possible sub-sequence lengths from 1 up to half of the remaining array
#         max_sub_len = (len(path) - i) // 2
#         for sub_len in range(1, max_sub_len + 1):
#             # Check if path[i : i+sub_len] == path[i+sub_len : i+2*sub_len]
#             if path[i : i+sub_len] == path[i+sub_len : i+2*sub_len]:
#                 repeat_len = sub_len
#                 found_repeat = True
#                 break
        
#         if found_repeat:
#             # We found a repeating sub-sequence of length repeat_len
#             # Add it once, and skip all consecutive repeats
#             sub_seq = path[i : i+repeat_len]
#             result.extend(sub_seq)  # keep only one instance
#             # Move i forward until the sub-sequence is no longer repeated
#             j = i + repeat_len
#             # While the next chunk is the same sub-sequence, skip it
#             while j + repeat_len <= len(path) and path[j : j+repeat_len] == sub_seq:
#                 j += repeat_len
#             i = j
#         else:
#             # No repeat found; just add this single element and move on
#             result.append(path[i])
#             i += 1
    
#     return result


# cleaned_path = remove_repeated_loops_keep_one(full_path_1)
# print(len(cleaned_path))
# print("full path: ",len(full_path_1))



1676
full path:  1703


In [31]:
# def remove_repeated_loops_keep_one(path, target_nodes):
#     """
#     Removes immediately repeated sub-sequences in the path,
#     keeping only one copy of that sub-sequence. Retains target nodes
#     even if they appear in repeating patterns.
    
#     Parameters:
#     - path: List of nodes in the path.
#     - target_nodes: Set of nodes that must not be removed.
    
#     Returns:
#     - List of nodes with repeating sub-sequences removed.
#     """
#     if not path:
#         return path
    
#     result = []
#     i = 0
#     while i < len(path):
#         # Step 1: Attempt to find any repeated sub-sequence starting at i
#         repeat_len = 1  # length of repeating pattern
#         found_repeat = False
#         max_sub_len = (len(path) - i) // 2

#         for sub_len in range(1, max_sub_len + 1):
#             # Check if path[i : i+sub_len] == path[i+sub_len : i+2*sub_len]
#             if path[i : i+sub_len] == path[i+sub_len : i+2*sub_len]:
#                 repeat_len = sub_len
#                 found_repeat = True
#                 break
        
#         if found_repeat:
#             # We found a repeating sub-sequence of length repeat_len
#             sub_seq = path[i : i+repeat_len]
#             # Retain the sub-sequence if any node in it is a target node
#             if any(node in target_nodes for node in sub_seq):
#                 result.extend(sub_seq)  # Keep the whole sub-sequence
#             else:
#                 result.extend(sub_seq)  # Keep only one instance of the sub-sequence

#             # Move i forward until the sub-sequence is no longer repeated
#             j = i + repeat_len
#             while j + repeat_len <= len(path) and path[j : j+repeat_len] == sub_seq:
#                 j += repeat_len
#             i = j
#         else:
#             # No repeat found; just add this single element and move on
#             result.append(path[i])
#             i += 1
    
#     return result



# cleaned_path = remove_repeated_loops_keep_one(full_path_1, modified)

# print(f"Original path length: {len(full_path_1)}")
# print(f"Cleaned path length: {len(cleaned_path)}")


Original path length: 1703
Cleaned path length: 1676


In [37]:
#final_path_to_cal_efficiency = construct_full_path(final_path= result['global_best_path'], path_details_df= path_details_df)
final_path_to_cal_efficiency = construct_full_path(final_path= bp_1, path_details_df= path_details_df)
full_path_df_to_cal_efficiency = create_ordered_dataframe(full_path = final_path_to_cal_efficiency, original_df = tilbury_df) 

In [38]:
total_distance_covered = full_path_df_to_cal_efficiency["weight"].astype(float).sum()
total_normal_distance = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'normal']['weight'].astype(float).sum()
total_target_distance = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'target']['weight'].astype(float).sum()
total_distinct_target_distance = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'target'].drop_duplicates()['weight'].astype(float).sum()

total_roads_covered = full_path_df_to_cal_efficiency.shape[0]
total_normal_roads = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'normal'].shape[0]
total_target_roads = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'target'].shape[0]
total_distinct_target_road = full_path_df_to_cal_efficiency[full_path_df_to_cal_efficiency['edge_stat'] == 'target'].drop_duplicates().shape[0]


print("total_distance_covered: ",total_distance_covered)
print("total_normal_distance: ", total_normal_distance )
print("total_target_distance: ", total_target_distance ) 
print("total_distinct_target_distance: ",total_distinct_target_distance )

print("total_roads_covered: ", total_roads_covered )
print("total_normal_roads: ", total_normal_roads ) 
print("total_target_roads: ", total_target_roads ) 
print("total_distinct_target_road: ", total_distinct_target_road)

total_distance_covered:  38766.080143492
total_normal_distance:  21307.611927341997
total_target_distance:  17458.46821615
total_distinct_target_distance:  15830.241045714
total_roads_covered:  983
total_normal_roads:  578
total_target_roads:  405
total_distinct_target_road:  355


total_distance_covered:  70371.462505067
total_normal_distance:  40199.073656042
total_target_distance:  30172.388849025003
total_distinct_target_distance:  23747.932792407002
total_roads_covered:  1702
total_normal_roads:  1013
total_target_roads:  689
total_distinct_target_road:  500

In [40]:
total_distinct_target_distance/total_distance_covered

np.float64(0.4083528947760162)

In [35]:
23747.932792407002/70371.462505067

0.3374653864938087

In [36]:
582/689

0.7256894049346879

In [21]:
from collections import Counter

# 1) Load your best path
full_top_df = result["full_top_df"]
best_df_1 = full_top_df[
    full_top_df["distinct_target_count"] == full_top_df["distinct_target_count"].max()
]
#bp_1 = best_df_1["path"].to_list()[0]
bp_1 = full_path_1
# 2) Count how many times each node is visited
node_counts = Counter(bp_1)
top_20_visited = node_counts.most_common(20)
print("Top 20 visited nodes:", top_20_visited)

### Advanced approach
def remove_sub_path_loops(path):
    visited_positions = {}
    clean_path = []
    i = 0
    
    while i < len(path):
        node = path[i]
        if node not in visited_positions:
            visited_positions[node] = len(clean_path)
            clean_path.append(node)
            i += 1
        else:
            loop_start_idx = visited_positions[node]
            clean_path = clean_path[:loop_start_idx]
            
            # Remove from visited_positions nodes that no longer exist
            for n, pos in list(visited_positions.items()):
                if pos >= loop_start_idx:
                    del visited_positions[n]

            i += 1
    
    return clean_path

clean_bp_1_subloops = remove_sub_path_loops(bp_1)

print("Original path length:", len(bp_1))
print("Clean path (subloops) length:", len(clean_bp_1_subloops))


Top 20 visited nodes: [('7416590683075160300', 10), ('8438794417920049598', 9), ('12565915507989070229', 7), ('13578739289540544228', 7), ('13005594061644765760', 7), ('6816013117868154408', 6), ('16497227840895154402', 6), ('11590977204505030050', 6), ('6204668632830655208', 6), ('9702561596159922947', 6), ('4739344667128443361', 6), ('12174193857474221806', 6), ('7112228690721332163', 6), ('6733184906911633693', 6), ('15416266433601332223', 6), ('385994481773363092', 6), ('13281099642587352679', 6), ('2239256113670293126', 6), ('6908990400828559958', 6), ('6217588509945271606', 6)]
Original path length: 1703
Clean path (subloops) length: 84


In [22]:
1703 - 84

1619